## Fetching data with NCBI API

### Background

Often, when conducting research you would want to download sequences or full genomes t combine with your own data, or when conducting a genomic data mining research. To get the data, the easiest approach would be to go to the NCBI web-page, search your genes and download via the browser. This works well. There are a few instances where that may not be possible, feasible, or time efficient:
- You are working from the HPC -- you'd have to download to your PC then copy to HPC via scp
- You are downloading large amount of data -- the browser maybe slow or terminate download in the midle
- You are searching for a large number of sequences or genomes -- it worn't be time efficient to download via the browser

Whatever the reason, it is a good time investment to learn to work with Biological APIs -- Your research life will be better because of it. 

In this session, we'll take you through a tutorial for downloading sequences and genomes using the API. We assume you can comfortably navigate the NCBI and other common Biological databases. 

### Requirements

This tutorial assumes:
- Some farmiliarity with the Linux command-line
- Some farmiliarity with Python
- Farmiliarity with Biological Databases

### But first, What are APIs?

![](https://cdn-images-1.medium.com/max/2400/1*OcmVkcsM5BWRHrg8GC17iw.png)


![](https://media2.govtech.com/images/940*603/api_infographic_smartfile_crop.jpg)

### Examples of Biological APIs

The most common Biological databases provide an API that allows the users to get the data programmatically. The most common API is the NCBI's E-Utilities. [This resource](https://www.programmableweb.com/news/101-biology-apis-biocatalogue-biogrid-and-synergizer/2013/10/22) contains a list of Biological databases that expose an API for programmatic data access. 

### Import the necessary modules

In [6]:
import urllib
import os
import sys
import time

### Entrenz Eutilis
The Entrez Programming Utilities (E-utilities) are a list of server-side programs that provide an interface to the Entrenz query and database system of NCBI. 
- Uses fxed URL syntax to send input parameters for serch and database query
- E-utility URL to posted NCBI, returns results that is processes the data as required using various programming languages
- can use any language that can send a URL to the E-utilities server and interpret the XML response

Any tool can be used to access the eutilis API, et EDirect

![](https://dataguide.nlm.nih.gov/images/edirect_context_small.png)


Databases will store data and create a user interface for data serch and Visualization; the APIs allows for automated exchange and integration of data from various database resources.

The API urls below can be used to download a whole genome from NCBI by just providing the ID and specifying the return type (Fasta or Genbank)

In [97]:
!curl "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&amp;id=CP000962&amp;rettype=fasta&amp;retmode=text" >Data/CP000962.fa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3955k    0 3955k    0     0   397k      0 --:--:--  0:00:09 --:--:--  498k


In [96]:
! curl "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=CP000962&rettype=gb&retmode=text" >Data/CP000962.gb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   327  100   327    0     0    458      0 --:--:-- --:--:-- --:--:--   457


In [7]:
url_template = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=%s&rettype=fasta&retmode=text"

Let's break down this template a bit:
- <http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?> This is command telling your computer program (or your browser) to talk to the NCBI API tool efetch.
- <db=nuccore> This command tells the NCBI API that you'd like it to look in this particular database for some data. You can lookup for other NCBI databases [here](https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi)
- <id=X51500.1> This command tells the NCBI API efetch the ID of the gene/genome you want to find.
- <rettype=gb&retmode=text> These two commands tells the NCBI how the data is returned. You'll note that in the two examples above this command varied slightly. In the first, we asked for only the FASTA sequence, while in the second, we asked for the Genbank file. Here's some elusive documentation on where to find these "return" objects.

Also, a useful command is also <version=1>. There are different versions of sequences and some times that is useful. For reproducibility, specify versions in your queries.

Now, let's use Python to access the API and download the sequences, given the IDs

In [19]:
ids = 'LN713527.1'

Remember string formatting? 

In [20]:
url_template % ids

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=LN713527.1&rettype=fasta&retmode=text'

In [101]:
!curl "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=LN713527.1&rettype=fasta&retmode=text"

>LN713527.1 Mesorhizobium sp. CCANP133 partial nifH gene for NifH, strain CCANP133
GGCAAGTCCACCACCTCCCAAAATACGCTCGCCGCCCTGGTCGACCTCGGGCAGAGAATTCTCATCGTCG
GCTGCGACCCCAAAGCCGACTCCACCCGCCTGATCCTGAACTCGAAGGCTCAGGATACTGTCCTGCATCT
GGCGGCACAGGAAGGTTCGGTGGAAGATCTCGAACTGCAGGACGTGCTCAAGATTGGCTACAGAGGCATC
AAATGCGTGGAGTCCGGCGGCCCGGAGCCGGGTGTTGGCTGCGCGGGCCGAGGCGTCATCACATCAATCA
ACTTCCTCGAGGAGAACGGCGCCTATGACGATGTCGACTATGTCTCCTACGACGTGCTCGGCGACGTGGT
TTGCGGCGGTTTCGCGATGCCGATCCGCGAGGGCAAGGCGCAGGAAATCTACATCGTCATGTCCGGGGAG
ATGATGGCGCTCTATGCCGCCAACAACATCGCCAAGGGCATCCTGAAATATGCCCATTCGGGGGGCGTGC
GGCTCGGAGGCCTGATCTGCAACGAACGTCAAACCGACCGTGAGCTCGACCTTGCTGAAGCCCTGGCTTC
CAGGCTCAATTCCAAGCTCATCCATTTCGTGCCGCGCGACAACATCGTTCAGCACGCCGAGCTCAGGAAG
ATGTCAGTTATCCAGTATGCGCCGGATTCCAAGCAGGCCGGGGAGTACCGCGCGCTGGCCGAGAAGATCC
ATGCCAATTCTGGCCAGGGCACCATCCCGACGCCGATCACCATGGACGA



In [86]:
url_template

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=%s&rettype=fasta&retmode=text'

As you can see below, we can use the urlopen function to dump the data

In [22]:
data = urllib.request.urlopen(url_template % ids).read()

But we can also write the data to a file

In [25]:
%%bash
mkdir Data

In [102]:
ls Data

CP000962.fa  CP000962.gb  LN713527.1.fa


In [28]:
with open('Data/%s.fa' % ids, 'w') as write_seq:
    write_seq.write(data.decode('utf-8')) # decode the bytes to string

Now, given a list of IDs, you can write a python function to download the fasta sequences into a single file or each as a separate file. But how do we get the ids in the first place? What if we want to search for sequences fitting a certain criteria?

In [16]:
template = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&term=biomol+trna[prop]"

In [54]:
xmls = urllib.request.urlopen(template)

In [55]:
xmls.read()

b'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">\n<eSearchResult><Count>853</Count><RetMax>20</RetMax><RetStart>0</RetStart><IdList>\n<Id>1485781757</Id>\n<Id>1485781756</Id>\n<Id>1485781755</Id>\n<Id>1485781754</Id>\n<Id>1485781753</Id>\n<Id>1485781752</Id>\n<Id>1485781751</Id>\n<Id>1485781750</Id>\n<Id>1485781749</Id>\n<Id>1485781748</Id>\n<Id>1485781747</Id>\n<Id>1485781746</Id>\n<Id>1485781745</Id>\n<Id>1485781744</Id>\n<Id>1395942016</Id>\n<Id>1395942015</Id>\n<Id>1395942014</Id>\n<Id>1395942013</Id>\n<Id>1395942012</Id>\n<Id>1395942011</Id>\n</IdList><TranslationSet/><TranslationStack>   <TermSet>    <Term>biomol trna[prop]</Term>    <Field>prop</Field>    <Count>853</Count>    <Explode>N</Explode>   </TermSet>   <OP>GROUP</OP>  </TranslationStack><QueryTranslation>biomol trna[prop]</QueryTranslation></eSearchResult>\n'

With such an output, you'll need to parse the XML to get the ids, which you can use to download the sequences. Alternatively, we can use the BioPython's wraper for the API, which allows us more flexibility. 

### Using BioPython
In the spirit of let's not re-invent the wheel, let's explore the Biopython Entenz module. But first, let's get it installed:

`conda install -c anaconda biopython `



In [57]:
from Bio import Entrez

#### Set the email address

In [58]:
Entrez.email = "calebkibet88@gmail.com"

In [59]:
disease = 'cancer'

In [63]:
handle = Entrez.esearch(db = "nucleotide", term="%s" % disease)

In [64]:
records = Entrez.read(handle)

In [82]:
records

DictElement({'Count': '9882836', 'RetMax': '20', 'RetStart': '0', 'IdList': ['1685641757', '1685639523', '1685630012', '1685629787', '1685629785', '1685622369', '1685622367', '1685622365', '1685617929', '1685617927', '1685614869', '1685614867', '1685613538', '1685609765', '1685609763', '1685609139', '1685609137', '1685609135', '1685609133', '1685604908'], 'TranslationSet': [DictElement({'From': 'cancer', 'To': '"Cancer"[Organism] OR cancer[All Fields]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"Cancer"[Organism]', 'Field': 'Organism', 'Count': '358', 'Explode': 'Y'}, attributes={}), DictElement({'Term': 'cancer[All Fields]', 'Field': 'All Fields', 'Count': '9882836', 'Explode': 'N'}, attributes={}), 'OR', 'GROUP'], 'QueryTranslation': '"Cancer"[Organism] OR cancer[All Fields]'}, attributes={})

Since the search results are returned as dictionary, you can easily access the ID lists, and for downloading the sequences. Let's try it out.

In [80]:
fetch = Entrez.efetch(db="nucleotide", id=records['IdList'][0], rettype="fasta", retmode="text")

In [81]:
print(fetch.read())

>XM_029511155.1 PREDICTED: Echeneis naucrates HIC ZBTB transcriptional repressor 2 (hic2), mRNA
CGTCCCGAAGGCTGTGAGATGAGCAAGAGGATGGTAGGCTAAAGACCGCTTGGGAACGATCCTACCGATG
TATTAAGCTTCAGGTGGCACTCGTGTAGGAAGAGAGAGAGAAGGAAAGAAGGAAGGTGGAGATGGAACTG
CCAAATCATGCCAAACACCTGCTGCTGCAACTCAACCAGCAGAGAGCCAAGGGCTTCCTGTGCGACGTTA
TCATCGTGGTGGAGAATGCACTCTTCCGTGCCCACAAGAACATCCTGGCCGCAAGCAGCATCTACTTCAA
ATCTTTGGTCCTTCACGATAACCTCATTAACCTTGATACAGAGATGGTAAATCCCTCTGTATTCAGACAA
GTTCTGGACTTCATCTACACTGGTAAGCTCCTGTCCTCGTCCGACCAGAGCAGTGAGCAGAACTTCAGTG
CCCTCTTGACTGCAGCCAGCTACCTCCAGCTCCATGACCTCGCTGCTCTGTGCAGAAAGAAGCTCAAGCG
CAGTGGTGGGAAACCCCTGCCAGGTAAACCCTCCACTCCAGGTCCCCTTAGCCGCTTACGCCTCAACAAC
CAGCGCCTTTCCTCCTCTACCCCTGCTGGACCCAACAACCACTATCCACCAACCCCCTCTGATGCCGACC
AGCCACAACCAGATGAAGGCCTTCGGGACAAGCTCTCAGATGATGAGATGTTTGTTGGCAGTTCTGGGAA
GAATGGGAATGGGGGGAATGGCAGCAGTAACGGCAACCTTAGCAGCGGAGGAAGTGCTGGGGAACCAGAC
CTCGGACTTGACCTGTCCAAGAAGAGCCCTCCCTCTGCGGGCACAGCCACGGATGCCCTCAGCCCACATA
GCAACTCCCAAGAATCCCCCCAATCTGCCTCAGTATCCACAACCAACAGTGC

And with that, you can convert this into a function or a standalone module to download your sequences. Remember to always inspect you data and ensure your code reteurns what you expected. 

### If you use the commandline curl or Biopython to download the data, in both instances you will have used the NCBI API...BioPython's Entrenz module is a tool that makes use of the API

## Why use an API? Let's revisit the reasons
- You have large amount of data to download
- You want to ensure easily reproducible data analysis
- You are working on the HPC, and that's the only direct way to get the data
- Why else?

## Be considerate with Entrenz resources...
- No more than 3 URL requests per second.
- At most 100 requests during the day (biopython
- Limit large jobs to either weekends or between 9:00PM -5:00 AM.
- Supply your email address and your tool name.
- Use Entrez history for large requests...otherwise you or your computer could be banned!

BioPython automates many of the requirements...

### An example in vectorbase

In [103]:
!wget -q --header='Content-type:application/json' 'https://www.vectorbase.org/rest/archive/id/AGAP004707?'  -O -

{"latest":"AGAP004707.2","version":2,"release":"95","peptide":null,"is_current":"1","id":"AGAP004707","type":"Gene","possible_replacement":[],"assembly":"AgamP4"}

In [106]:
import requests, sys
 
server = "https://www.vectorbase.org/rest"
ext = "/sequence/id"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
r = requests.post(server+ext, headers=headers, data='{ "ids" : ["AGAP004707", "AGAP004727" ] }')
 
if not r.ok:
    r.raise_for_status()
    sys.exit()

In [108]:
decoded = r.json()
print(repr(decoded))

[{'desc': 'chromosome:AgamP4:2L:2358158:2431617:1', 'query': 'AGAP004707', 'version': 2, 'id': 'AGAP004707', 'seq': '...', 'molecule': 'dna'}, {'desc': 'chromosome:AgamP4:2L:2812195:2814547:1', 'query': 'AGAP004727', 'version': 4, 'id': 'AGAP004727', 'seq': 'CTACGATACAGCCCGGTCATGGAAATATCAGAGTCTACTAAAGTAGTGTTTTACGTAAGCAGAATATTTGGTCTTGCACCTTATGTAGTGAAGCGCACTCCTATTGGCCAAATAGTGGATTACAAGCGCAGCGTGTTTCTCATAGCTTATAGTGTAGGTCTTGTGTTTTGCTTAGGTACGTACAACGTTTATACATGTTGTACCCATTTGTCTGCGCCTATAAAATAAGCCTTTCCCTACCACTTCTTCACATCATTTTGTGTTATCAACTATCAAACAATTCTACCAGCTGGCCTCACCTACAAAGGAATATTTTTCGATGTAACTTCCAAAAAACCTATACGGTAAGACATACGATGTGCCTTTACATGTCGTACATCCCCGACCATAGGAACAGCCCGTTAACAATGTTAAAAAGCTTCGTTTTCAGAATGAAAACAGCTACTTCAAAGGTGGTGACCATTCTAGACGTGTCGGTTGTAGTGTCTGCCTGCATTTGTGGCACCCTCAGTGGAATAGGGGGACTGCATTTTGTGCGTGAGCTTAATCGACGGTTAACCGAAGTGAGATCCGCTTAAGTGTTTTCTAAATTTTCCCACAGCTGTACAGCACTTTACCATGTATTTGTAATATTTTTTTCAATCATTCTACACTCCTTTAAATCATAATATGTACGAGGCCGATGAAATACTGTCATTGTACGTCAAGGAAAGTACCCGAAGTCGGAGGCAACGACGCAAA